In [1]:
import os
import tensorflow as tf
#download mnist data to /tmp/data/
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print("Total size of {} training data".format(int(mnist.train.num_examples)))

tb_logdir = "/tmp/tf_tutorial/03_LogisticRegression"

#Data parameters
image_size = 28*28
n_class = 10

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Total size of 55000 training data


In [2]:
#Training parameters
learning_rate = 1e-2
total_epochs = 30
batch_size = 100

#names are for tensorboard.
x = tf.placeholder(tf.float32, [None, image_size], name='InputImage')
y = tf.placeholder(tf.float32, [None, n_class], name='Label')
W = tf.Variable(tf.zeros([image_size, n_class]), name='Weight')
b = tf.Variable(tf.zeros([n_class]), name='Bias')

In [3]:
with tf.name_scope("Model"):
    pred_y = tf.nn.sigmoid(tf.matmul(x, W) + b)
with tf.name_scope("Loss"):
    loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred_y), reduction_indices=1))
with tf.name_scope("SGD"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
with tf.name_scope("Accuracy"):
    correct = tf.equal(tf.argmax(pred_y, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
#Adding image summary for input image, which requires a 4D shape.
input_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image("input_image", input_image)
tf.summary.scalar("loss_summary", loss)
merged_summary_op = tf.summary.merge_all()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(tb_logdir, graph=tf.get_default_graph())
    for epoch in range(total_epochs):
        sum_loss = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for batch in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            summary, _, l = sess.run([merged_summary_op, optimizer, loss], feed_dict={x:batch_x, y:batch_y})
            train_writer.add_summary(summary, epoch*total_batch+batch)
            sum_loss += l
        avg_loss = sum_loss/total_batch
        if epoch%5 == 0:
            print("Epoch {:02}  Average loss: {:.5f}".format(epoch, avg_loss))
    print("Training Complete!")
    train_loss, train_acc = sess.run([loss, accuracy], feed_dict={x:mnist.train.images, y:mnist.train.labels})
    print("Final Training loss: {:.5f}   acc:{:.2f}%".format(train_loss, train_acc*100))
    test_loss, test_acc = sess.run([loss, accuracy], feed_dict={x:mnist.test.images[:3000], y:mnist.test.labels[:3000]})
    print("Testing loss:        {:.5f}   acc:{:.2f}%".format(test_loss, test_acc*100))
    
#You can also open tensoboard through terminal beforehand to make sure correctness.
#(e.g. whetherthe loss is reducing, correct input image...)
print("Starting tensorboard...")
print("Open http://0.0.0.0:6006/ for viewing.")
os.system("tensorboard --logdir={}".format(tb_logdir))

Epoch 00  Average loss: 0.15464
Epoch 05  Average loss: 0.01177
Epoch 10  Average loss: 0.00657
Epoch 15  Average loss: 0.00463
Epoch 20  Average loss: 0.00360
Epoch 25  Average loss: 0.00296
Training Complete!
Final Training loss: 0.00256   acc:68.09%
Testing loss:        0.00329   acc:68.10%
Starting tensorboard...
Open http://0.0.0.0:6006/ for viewing.
